In [1]:
# RUN JUST ONCE TO DOWNLOAD THE DRIVER
from webdriver_manager.chrome import ChromeDriverManager

# Chrome driver installation
CHROMEDRIVER_PATH = ChromeDriverManager().install()

print(CHROMEDRIVER_PATH) # Chrome driver path

In [255]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time

def start_driver():
    """ Initialize Chrome driver and open main page """
    
    # Threads URL
    URL = "https://www.threads.net/?hl=en"
    # Chrome driver PATH
    CHROMEDRIVER_PATH = r"C:\Users\lucal\.wdm\drivers\chromedriver\win64\131.0.6778.85\chromedriver-win32/chromedriver.exe"
    
    # Initialize the Chrome driver
    driver = webdriver.Chrome(service=Service(CHROMEDRIVER_PATH))
    
    # # Navigate to the URL + explicit wait
    driver.get(URL)
    time.sleep(2)
    
    return driver

In [256]:
# Import the By class for locating elements
from selenium.webdriver.common.by import By
# Import wait and EC to form an implicit wait
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
# timeout exception
from selenium.common.exceptions import TimeoutException

def accept_cookies(driver):
    """ accept coookies """

    # CSS SELECTOR
    ACCEPT_COOKIE_SELECTOR = 'body > div.__fb-dark-mode.x1n2onr6.xzkaem6 > div:nth-child(2) > div > div > div.x9f619.x1n2onr6.x1ja2u2z > div > div.x1uvtmcs.x4k7w5x.x1h91t0o.x1beo9mf.xaigb6o.x12ejxvf.x3igimt.xarpa2k.xedcshv.x1lytzrv.x1t2pt76.x7ja8zs.x1n2onr6.x1qrby5j.x1jfb8zj > div > div > div > div.x6s0dn4.x78zum5.xl56j7k.x1n2onr6.x1vjfegm > div > div > div > div.x6bh95i.x13fuv20.x178xt8z.x1p5oq8j.xxbr6pl.xwxc41k.xbbxn1n > div > div.x1i10hfl.x1ypdohk.x2lah0s.xe8uvvx.xdj266r.x11i5rnm.xat24cr.x1mh8g0r.x2lwn1j.xexx8yu.x18d9i69.x1n2onr6.x16tdsg8.x1hl2dhg.xggy1nq.x1ja2u2z.x1t137rt.x1q0g3np.x1lku1pv.x1a2a7pz.x6s0dn4.x1a2cdl4.xnhgr82.x1qt0ttw.xgk8upj.x9f619.x3nfvp2.x1s688f.x90ne7k.xl56j7k.x193iq5w.x1swvt13.x1pi30zi.x12w9bfk.x1g2r6go.x11xpdln.xz4gly6.x87ps6o.xuxw1ft.x19kf12q.x111bo7f.x1vmvj1k.x45e8q.x3tjt7u.x35z2i1.x13fuv20.xu3j5b3.x1q0q8m5.x26u7qi.x178xt8z.xm81vs4.xso031l.xy80clv.xteu7em.x11r8ahe.x1iyjqo2.x15x72sd > div'
    # XPATH
    ACCEPT_COOKIE_XPATH = "//div[contains(text(), 'Allow all cookies')]"

    # TIMEOUT (15 seconds)
    TIMEOUT = 15
    
    try:
        # implicit wait
        button = WebDriverWait(driver, TIMEOUT).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ACCEPT_COOKIE_SELECTOR)) # with CSS_SELCTOR
        )
        # click to accept cookies
        button.click()
        print("CSS cookies button click\n")
    
    except TimeoutException:
        try: 
            button = WebDriverWait(driver, TIMEOUT).until(
                EC.presence_of_element_located((By.XPATH, ACCEPT_COOKIE_XPATH)) # with XPATH
            )
            button.click()
            print("XPTAH cookies button click\n")

        except TimeoutException:    
            print("ERROR: Cookies button missing")
            driver.quit() # cleaner termination ensuring a clean state for future executions

    

In [257]:
#XPATH
'''
POST CONTAINER RAW XPATH:
POST_XPATH 1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div
POST_XPATH 2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div
POST_XPATH 3RD POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[3]/div/div/div/div
POST_XPATH 45TH POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[45]/div/div/div/div
POST_XPATH 46TH POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[46]/div/div/div/div
POST_XPATH $N$ POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[$N$]/div/div/div/div


IMAGE RAW XPATH:
1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[1]/div/a/div/img
2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[1]/div/a/div/img

USERNAME RAW XPATH:
1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a/span/
2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a/span/

USERNAME LINK RAW XPATH:
1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a
2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[1]/div/div/a

POST LINK RAW XPATH:
1ST POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[1]/div/div/div/div/div[2]/div/div[1]/div/span[2]/span/a
2ND POST : //*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[2]/div/div/div/div/div[2]/div/div[1]/div/span[2]/span/a
'''

IMAGE_PROFILE_REL_XPATH = './/div[1]/div/a/div/img'
USERNAME_REL_XPATH      = './/div[2]/div/div[1]/div/span[1]/div/div/a/span'
USERNAME_LINK_REL_XPATH = './/div[2]/div/div[1]/div/span[1]/div/div/a'
POST_LINK_REL_XPATH     = './/div[2]/div/div[1]/div/span[2]/span/a'

In [258]:
from selenium.common.exceptions import NoSuchElementException

def grab_post(driver, post_number):
    """ Grab post and return a list of dict: image profile link, username, username link and post link """
    
    # Post XPATH
    POST_XPATH = f'//*[@id="barcelona-page-layout"]/div/div/div[2]/div[1]/div[1]/div/div/div[1]/div[{post_number}]/div/div/div/div'
  
    # class to handle each post
    class Post:
        def __init__(self, imgProfile, username, username_link, post_link):
            self._img_profile = img_profile
            self._username = username
            self._username_link = username_link
            self._post_link = post_link
        
        #getter
        @property
        def single_post(self):
            return {
                "img_profile": self._img_profile, 
                "username": self._username,
                "username_link": self._username_link,
                "post_link": self._post_link
                }
    
    # retrieves a specific post on the page
    post = driver.find_element(By.XPATH, POST_XPATH)
    
    # build the class and add element to list
    try:
        # image link
        img_profile = post.find_element(By.XPATH, IMAGE_PROFILE_REL_XPATH).get_attribute('src') 
        # username visible text
        username = post.find_element(By.XPATH, USERNAME_REL_XPATH).text
        # username link profile
        username_link = post.find_element(By.XPATH, USERNAME_LINK_REL_XPATH).get_attribute('href')
        # post content
        post_link = post.find_element(By.XPATH, POST_LINK_REL_XPATH).get_attribute('href')
        
        # new instance of the class Post
        new_post_obj = Post(img_profile, username, username_link, post_link)

        # create dictonary
        new_post = new_post_obj.single_post
    
    # incomplete post
    except NoSuchElementException:
        print("NoSuchElementException")  

    # other exception
    except Exception as e:
        print("Exception in grab_post function:", e)
        
    # return the new post collected   
    return new_post

In [259]:
def scroll(driver, px):
    """ scroll page """
    
    js = f"window.scrollBy(0, {px});"
    driver.execute_script(js)

In [260]:
def scroll_to_bottom(driver):
    """ auto-scroll to the bottom of the page """
   
    # post number to retrieve
    post_number = 1
    
    # list of posts
    list_to_push_to_db = []
    
    old_position = driver.execute_script("return window.pageYOffset;")
    
    while True:
        # Wait for the page to load
        time.sleep(3)

        try:
            # retrieve post
            post = grab_post(driver, post_number)
            
            ########################## save to db: TODO######################
            list_to_push_to_db.append(post)
          
            ########################## debug ################
            print("post_number:", post_number)
            print(f"list_to_push_to_db[{post_number-1}]", list_to_push_to_db[post_number-1])

        except NoSuchElementException:
            print(f"Post {post_number} not found")
            
        except Exception as e:
            print(f"Post number {post_number} Error: {e}")
            
        finally:
            # update post position
            post_number += 1
        
        # scroll down
        scroll(driver, 500)
        # update position
        new_position = driver.execute_script("return window.pageYOffset;")

        # Exit the loop if the page hasn't scrolled, meaning end of page
        if new_position == old_position:
            break 

        # update new position
        old_position = new_position 

In [261]:
from selenium.common.exceptions import InvalidSessionIdException

def close_session(driver):
    """ close active browser tab """
    
    try:
        driver.close() # exit driver
    except InvalidSessionIdException:
        print("Browser tab already closed!")

In [262]:
############################
####### RUN SCRAPER ########
############################
        
# start web driver
driver = start_driver() 
# accept initial cookies
accept_cookies(driver)

print("Website:", driver.title) # fetches the title of the current webpage
print("Current URL:", driver.current_url) # capture the final URL after all redirects have been resolved

# infinite scroll page
scroll_to_bottom(driver)

# stop
close_session(driver)

CSS cookies button click

Website: Threads
Current URL: https://www.threads.net/?hl=en
post_number: 1
list_to_push_to_db[0] {'img_profile': 'https://scontent.cdninstagram.com/v/t51.2885-19/465192065_908279924153519_6227935515441608614_n.jpg?stp=dst-jpg_s150x150_tt6&_nc_ht=scontent.cdninstagram.com&_nc_cat=111&_nc_ohc=eiyNjvTnV4EQ7kNvgGz-07V&_nc_gid=fc16dcb1c05443d682da80bb8afc4ad8&edm=APs17CUBAAAA&ccb=7-5&oh=00_AYDLFq2cL-Mu3h3ZEKRRybRWCjjo0LtILaKxsIOILWbxcw&oe=6753E756&_nc_sid=10d13b', 'username': 'yoko_photo__', 'username_link': 'https://www.threads.net/@yoko_photo__', 'post_link': 'https://www.threads.net/@yoko_photo__/post/DDEYz2DvR6o'}
post_number: 2
list_to_push_to_db[1] {'img_profile': 'https://scontent.cdninstagram.com/v/t51.2885-19/358167345_3599814276922619_5620889529209829492_n.jpg?stp=dst-jpg_s150x150&_nc_ht=scontent.cdninstagram.com&_nc_cat=1&_nc_ohc=czJFu26LGA0Q7kNvgFSp053&_nc_gid=fc16dcb1c05443d682da80bb8afc4ad8&edm=APs17CUBAAAA&ccb=7-5&oh=00_AYA61pTSaY9o-YmJMRrffr2iRP2zE

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=131.0.6778.86)
Stacktrace:
	GetHandleVerifier [0x004633E3+25059]
	(No symbol) [0x003ECDE4]
	(No symbol) [0x002CBEC3]
	(No symbol) [0x002AD93B]
	(No symbol) [0x0033800F]
	(No symbol) [0x0034AE49]
	(No symbol) [0x00331C96]
	(No symbol) [0x00303FAC]
	(No symbol) [0x00304F3D]
	GetHandleVerifier [0x00755543+3113795]
	GetHandleVerifier [0x0076A20A+3198986]
	GetHandleVerifier [0x007629E2+3168226]
	GetHandleVerifier [0x00503250+680016]
	(No symbol) [0x003F572D]
	(No symbol) [0x003F29D8]
	(No symbol) [0x003F2B75]
	(No symbol) [0x003E57D0]
	BaseThreadInitThunk [0x772D7BA9+25]
	RtlInitializeExceptionChain [0x77ABC0CB+107]
	RtlClearBits [0x77ABC04F+191]


In [18]:
# ENV VARIABLES
# https://saturncloud.io/blog/how-to-set-environment-variables-in-jupyter-notebooks-a-guide-for-data-scientists/

# add driver settings
# cookise
# headers
# proxy or VPN
# beware honeypot